연산자 오버로딩은 사용자 정의 객체가 +와 | 같은 중위 연산자, -와 ~같은 단항 연산자를 사용할 수 있게 해준다. 파이썬은 여기에서 더 나아가 함수 호출(()), 속성 접근(.), 항목 접근 / 슬라이싱([])도 연산자로 구현되어 있지만, 이 장에서는 단항 연산자와 중위 연산자만 다룬다.

- 파이썬이 다른 자료형의 피연산자로 중위 연산자를 지원하는 방법
- 다양한 자료형의 피연산자를 다루기 위한 덕 타이핑이나 명시적인 자료형 검사의 사용
- ==, >, <= 등 향상된 비교 연산자의 별난 행동
- 피연산자를 처리할 수 없다고 중위 연산자 메서드가 알려주는 방법

- +=과 같은 계산 할당 연산자의 기본 처리 방식 및 오버로딩 방법

# 1. 연산자 오버로딩 기본 지식  
  
  파이썬은 다음과 같은 제한을 두어 융통성, 사용성, 안전성을 적절히 유지한다.


 - 내장 자료형에 대한 연산자는 오버로딩할 수 없다.
 - 새로운 연산자를 생성할 수 없으며, 기존 연산자를 오버로딩만 할 수 있다.
 - is, and, or, not 연산자는 오버로딩할 수 없다(그러나 $, |, ~ 비트 연산자는 가능)

# 2. 단항 연산자  
  

"-" (__neg __) : 단항 산술 부정. x가 -2면 -x는 2다.

"+" (__pos __) : 단항 산술 덧셈. 일반적으로 x와 +x는 동일하다.

"~" (__invert __) : 정수형의 비트 반전. ~x는 -(x+1)로 정의된다.  

단항 연산자는 구현하기 쉽다. 단지 self 인수 하나를 받는 적절한 특별 메서드를 구현하면 된다. 클래스에 논리적으로 합당한 연산을 수행해야 하지만, 특히 "언제나 새로운 객체를 반환해야 한다"는 연산자의 핵심 규칙을 지켜야한다.

In [1]:
from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools # chain() 함수를 사용하기 위함

class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'
    
    def __init__(self, components):
        # 벡터 요소를 배열로 저장
        self._components = array(self.typecode, components)
    
    """시퀀스 프로토콜 구현"""
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self) # 객체의 클래스를 가져옴
        if isinstance(index, slice): # index가 슬라이스이면
            return cls(self._components[index]) # Vector 객체를 생성
        elif isinstance(index, numbers.Integral): # index가 정수형이면
            return self._components[index] # 해당 항목 반환
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls)) 
    """시퀀스 프로토콜 구현 종료"""
    
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1: # name이 한글자이면 
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components): # 포지션이 범위 내에 있으면 배열 항목 반환
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}' 
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}' # xyzt 중 하나는 구체적으로 오류 발생
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}" # 그외 소문자면 일반적 메세지 오류 발생
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        # 에러가 발생하지 않을 때는 정상적으로 __setattr__() 메서드 호출
        super().__setattr__(name, value) 
        
    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1]<0):
            return math.pi * 2 - a
        else:
            return a
        
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'): # 초구면좌표
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                    self.angles()) 
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))
    
    def __iter__(self):
        return iter(self._components) # 반복할 수 있도록 구현
    
    def __repr__(self):
        components = reprlib.repr(self._components) # 제한된 길이로 출력
        components = components[components.find('['):-1] # 문자열 중 앞에 나오는 "array('d'," 를 제거
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
               bytes(self._components))
    
    def __eq__(self, other):
        if len(self) != len(other): # 길이가 다르면 False
            return False
        for a, b in zip(self, other): # 제너레이터로부터 하나씩 비교
            if a != b:
                return False
        return True
    
    def __hash__(self):
        hashes = (hash(x) for x in self._components) # 제너레이터 표현식 이용
        return functools.reduce(operator.xor, hasehs, 0) # 초기값을 0으로 함
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv) # 언패킹할 필요가 없음
    
    # -x 구현
    def __neg__(self):
        return Vector(-x for x in self)
    
    # +x 구현
    def __pos__(self):
        return Vector(self)

In [2]:
v1 = Vector([1,2,3])
v1, +v1, -v1

(Vector([1.0, 2.0, 3.0]), Vector([1.0, 2.0, 3.0]), Vector([-1.0, -2.0, -3.0]))

In [6]:
# 산술 콘텍스트의 정밀도를 변경하면 x와 +x가 달라질 수 있다.

import decimal 
ctx = decimal.getcontext()
ctx.prec = 40
one_third = decimal.Decimal('1') / decimal.Decimal('3')
one_third

Decimal('0.3333333333333333333333333333333333333333')

In [5]:
one_third == +one_third

True

In [10]:
ctx.prec = 28
one_third == +one_third

False

In [11]:
+one_third

Decimal('0.3333333333333333333333333333')

In [18]:
# 단항 연산자는 +는 합계가 0이거나 음수인 항목을 제외하고 Counter 객체를 새로 생성한다

from collections import Counter
ct = Counter('abracadabra')
ct

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [16]:
ct['r'] = -3
ct['d'] = 0
ct

Counter({'a': 5, 'b': 2, 'r': -3, 'c': 1, 'd': 0})

In [17]:
+ct

Counter({'a': 5, 'b': 2, 'c': 1})

# 3 벡터를 더하기 위해 + 오버로딩하기  

짧은 쪽에 벡터의 빈 공간을 0으로 채워서 더하도록

In [19]:
from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools # chain() 함수를 사용하기 위함

class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'
    
    def __init__(self, components):
        # 벡터 요소를 배열로 저장
        self._components = array(self.typecode, components)
    
    """시퀀스 프로토콜 구현"""
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self) # 객체의 클래스를 가져옴
        if isinstance(index, slice): # index가 슬라이스이면
            return cls(self._components[index]) # Vector 객체를 생성
        elif isinstance(index, numbers.Integral): # index가 정수형이면
            return self._components[index] # 해당 항목 반환
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls)) 
    """시퀀스 프로토콜 구현 종료"""
    
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1: # name이 한글자이면 
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components): # 포지션이 범위 내에 있으면 배열 항목 반환
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}' 
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}' # xyzt 중 하나는 구체적으로 오류 발생
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}" # 그외 소문자면 일반적 메세지 오류 발생
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        # 에러가 발생하지 않을 때는 정상적으로 __setattr__() 메서드 호출
        super().__setattr__(name, value) 
        
    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1]<0):
            return math.pi * 2 - a
        else:
            return a
        
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'): # 초구면좌표
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                    self.angles()) 
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))
    
    def __iter__(self):
        return iter(self._components) # 반복할 수 있도록 구현
    
    def __repr__(self):
        components = reprlib.repr(self._components) # 제한된 길이로 출력
        components = components[components.find('['):-1] # 문자열 중 앞에 나오는 "array('d'," 를 제거
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
               bytes(self._components))
    
    def __eq__(self, other):
        if len(self) != len(other): # 길이가 다르면 False
            return False
        for a, b in zip(self, other): # 제너레이터로부터 하나씩 비교
            if a != b:
                return False
        return True
    
    def __hash__(self):
        hashes = (hash(x) for x in self._components) # 제너레이터 표현식 이용
        return functools.reduce(operator.xor, hasehs, 0) # 초기값을 0으로 함
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv) # 언패킹할 필요가 없음
    
    # -x 구현
    def __neg__(self):
        return Vector(-x for x in self)
    
    # +x 구현
    def __pos__(self):
        return Vector(self)

    def __add__(self, other):
        pairs = itertools.zip_longest(self, other, fillvalue=0.0)
        return Vector(a + b for a, b in pairs)

In [22]:
v1 = Vector([3,4,5])
v2 = Vector([6,7,8])
v1+v2

Vector([9.0, 11.0, 13.0])

In [23]:
v1+v2 == Vector([3+6, 4+7, 5+8])

True

In [25]:
v1 = Vector([1,2])
v2 = Vector([1,4,6,2])
v1 + v2

Vector([2.0, 6.0, 6.0, 2.0])

# 4, 벡터를 스칼라와 곱하기 위해 * 오버로딩하기

![](https://raw.githubusercontent.com/lih0905/Fluent_Python/1d739915cc054f11c0081245f53f7e66f7368915//images/13_1.PNG)
![](https://raw.githubusercontent.com/lih0905/Fluent_Python/1d739915cc054f11c0081245f53f7e66f7368915//images/13_2.PNG)

# 5. 향상된 비교 연산자

![](https://raw.githubusercontent.com/lih0905/Fluent_Python/1d739915cc054f11c0081245f53f7e66f7368915//images/13_3.PNG)